# Mutual Information-based Registration

## Introduction
In this exercise, you will implement 3D rigid image registration using mutual information (MI). You will work with a T1-weighted and a T2-weighted MRI scan of the same patient, where the patient is lying in a different position in the scanner between the two image acquisition (for the purpose of this exercise, the movement is simulated).

### Instructions for your report
Your report should be structured as follows:
* **Introduction**: a short summary of what your report is about, perhaps with a recap of some of the equations you'll be using in your solution.
* **Task 1, ..., 5**: for each specific task listed below, your code (in code cells), your results (as figures) as well as explanations of what is computed and what is shown in the figures (in Markdown cells). 
* **Conclusion**: a short summary of your findings.

This introduction should **not** be part of your report.

### Input data and code hints
Import Python libraries:

In [36]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
#%matplotlib notebook
%matplotlib tk
plt.ion()
np.set_printoptions( suppress=True )
import nibabel as nib
import scipy
from scipy.ndimage import map_coordinates


Read the two 3D scans you'll be working with in this exercise:

In [37]:
T1_fileName = 'IXI014-HH-1236-T1.nii.gz'
T2_fileName = 'IXI014-HH-1236-T2_rotated.nii.gz'
T1 = nib.load( T1_fileName )
T2 = nib.load( T2_fileName )
T1_data = T1.get_fdata()
T2_data = T2.get_fdata()

Below is code to define a simple interactive viewer class that can be used to visualize 2D cross-sections of a 3D array along three orthogonal directions. It takes a 3D volume as input and shows the location a "linked cursor" in all three cross-sections.

The initial location of the cursor is in the middle of the volume in each case. It can be changed by clicking on one of the cross-sections. The viewer also displays the voxel index $\mathbf{v}$ of the cursor.

In [38]:
class Viewer:
    def __init__(self, data ):
        self.fig, self.ax = plt.subplots()
        self.data = data
        self.dims = self.data.shape
        self.position = np.round( np.array( self.dims ) / 2 ).astype( int )
        self.draw()
        self.fig.canvas.mpl_connect( 'button_press_event', self )
        self.fig.show()

    def __call__(self, event):
        print( 'button pressed' )
        if event.inaxes is None: return
      
        x, y = round( event.xdata ), round( event.ydata )

        #
        if ( x > (self.dims[0]-1) ) and ( y <= (self.dims[1]-1) ): return # lower-right quadrant
          
        #
        if x < self.dims[0]:
          self.position[ 0 ] = x
        else:
          self.position[ 1 ] = x - self.dims[0]
        
        if y < self.dims[1]:
          self.position[ 1 ] = y
        else:
          self.position[ 2 ] = y -self.dims[1]
        
        print( f"  voxel index: {self.position}" )
        print( f"  intensity: {self.data[ self.position[0], self.position[1], self.position[2] ]}" )

        self.draw()

    def draw( self ):
        #
        # Layout on screen is like this:
        #
        #     ^            ^
        #  Z  |         Z  |
        #     |            |
        #     ----->        ---->  
        #       X             Y
        #     ^
        #  Y  |
        #     |
        #     ----->  
        #       X
        #
        dims = self.dims
        position = self.position
        
        xySlice = self.data[ :, :, position[ 2 ] ]
        xzSlice = self.data[ :, position[ 1 ], : ]
        yzSlice = self.data[ position[ 0 ], :, : ]
        
        kwargs = dict( vmin=self.data.min(), vmax=self.data.max(), 
                       origin='lower', 
                       cmap='gray',
                       picker=True )

        self.ax.clear()

        self.ax.imshow( xySlice.T, 
                        extent=( 0, dims[0]-1, 
                                 0, dims[1]-1 ), 
                        **kwargs )
        self.ax.imshow( xzSlice.T, 
                        extent=( 0, dims[0]-1, 
                                 dims[1], dims[1]+dims[2]-1 ), 
                        **kwargs )
        self.ax.imshow( yzSlice.T, extent=( dims[0], dims[0]+dims[1]-1, 
                                            dims[1], dims[1]+dims[2]-1 ), 
                        **kwargs )

        color = 'g'
        self.ax.plot( (0, dims[0]-1), (position[1], position[1]), color )
        self.ax.plot( (0, dims[0]+dims[1]-1), (dims[1]+position[2], dims[1]+position[2]), color )
        self.ax.plot( (position[0], position[0]), (0, dims[1]+dims[2]-1), color )
        self.ax.plot( (dims[0]+position[1], dims[0]+position[1]), (dims[1]+1, dims[1]+dims[2]-1), color )

        self.ax.set( xlim=(1, dims[0]+dims[1]), ylim=(0, dims[1]+dims[2]) )

        self.ax.text( dims[0] + dims[1]/2, dims[1]/2, 
                      f"voxel index: {position}",  
                      horizontalalignment='center', verticalalignment='center' )
  
        self.ax.axis( False )

        self.fig.canvas.draw()

### Task 1: Resample the T2-weighted scan to the image grid of the T1-weighted scan

In this task you perform the resampling just like in the landmark-based registration exercise. Find the voxel-to-voxel transformation between $\mathbf{v}_{T1}$ and $\mathbf{v}_{T2}$:

$$
\begin{pmatrix} \mathbf{ v_{T2}} \\ 1 \end{pmatrix} = \mathbf{M}_{T2}^{-1} \cdot \mathbf{M}_{T1} \cdot \begin{pmatrix} \mathbf{ v_{T1}} \\ 1 \end{pmatrix}
.
$$
At the location $\mathbf{v}_{T2}$, you should then use cubic B-spline interpolation to determine the intensity in the T2-weighted scan, and store it at index $\mathbf{v}_{T1}$ in the newly created image.

> ***Hints:***
> - you can create a coordinate grid in 3D with the function
> 
>        V1,V2,V3 = np.meshgrid( np.arange( T1_data.shape[0] ), 
>                                np.arange( T1_data.shape[1] ), 
>                                np.arange( T1_data.shape[2] ), indexing='ij' )
>   
>
> - the following SciPy function interpolates the T2-weighted volume at voxel coordinates $(1.1,2.2,3.3)^T$ 
> and $(6.6,7.7,8.8)^T$ using cubic interpolation:   
>
>        scipy.ndimage.map_coordinates( T2_data, np.array( [ [1.1,2.2,3.3], [6.6,7.7,8.8] ] ).T )

In [39]:
M_1 = T1.affine
M_2 = T2.affine
V1, V2, V3 = np.meshgrid(np.arange(T1_data.shape[0]),
                         np.arange(T1_data.shape[1]),
                         np.arange(T1_data.shape[2]),
                         indexing= 'ij')
V1, V2, V3 = V1.flatten(), V2.flatten(), V3.flatten()
print(V3)
V_T1 = np.column_stack([V1, V2, V3, np.ones(len(V3))])
V_T2 = np.linalg.solve(M_2, M_1 @ V_T1.T)

T2_int = map_coordinates(T2_data, V_T2[:3], order = 3)
T2_re  = T2_int.reshape(T1_data.shape)
Viewer(T2_re/T2_re.max() + T1_data/T1_data.max())
Viewer(T2_re)

[  0   1   2 ... 147 148 149]


Once you have created the resampled T2-weighted data, visualize the T1-weighted and resampled T2-weighted volumes overlaid as follows:
    
        Viewer( T2_data_resampled / T2_data_resampled.max() + T1_data / T1_data.max() )

 Can you determine around what axis (in world coordinates) the rotation was simulated?

 > ***Hint***
 > - The T2-weighted volume was rotated around a single axis only.
 >

firstly find the robust max 98% of the full intensities, then we can do the joint histogram and single histograms


In [40]:
T1_rm= np.percentile(T1_data, 98).astype(int)
T2_rm= np.percentile(T2_re, 98).astype(int) # The resampled T2

print("T1 robust max and max", T1_rm, T1_data.max())
print("T2 robust max and max", T2_rm, T2_re.max())

T1 robust max and max 1638 4479.412590265274
T2 robust max and max 891 2505.8759338642885


### Task 2: Compute and visualize the joint histogram

Compute the joint histogram 
$$ 
\mathbf{H} = 
\begin{pmatrix}
h_{1,1} & ... & h_{1,B} \\
\vdots & ... & \vdots \\
h_{B,1} & ... & h_{B,B} \\
\end{pmatrix} 
$$
of the T1-weighted scan and the resampled T2-weighted scan you just created, using $B=32$ bins. In order not to let the registration process be dominated by the (extremely numerous) background voxels, ignore all voxels with an intensity lower than 10 in either image when computing the histogram. Visualize the joint histogram using a 3D bar plot, and label the axes.

> ***Hints:***
>
> - To compute a 2D histogram, you can use the function
>
>           np.histogram2d(x,y,bins,range)
> 
> - A 3D bar plot can be created as follows:
>
>           fig, ax = plt.subplots( subplot_kw=dict( projection='3d') )
>           plt.bar3d(xpos, ypos, zpos, dx, dy, dz)
>
>   where xpos, ypos, and zpos are the anchor points of the bars and dx, dy, and dz are the dimensions of the bars.
>
> - A useful "trick" to verify that your axes are labeled correctly, is to use a different number of bins for the two scans.
>

In [41]:
bins = 32
lb   = 10

H, xedges, yedges = np.histogram2d(T1_data.flatten(), T2_re.flatten(), bins = bins, range=[[lb, T1_data.max()], [lb, T2_re.max()]])
jointProb = H / np.sum(H)
xpos, ypos = np.meshgrid(xedges[:-1]+xedges[1:], yedges[:-1]+yedges[1:]-xedges[0])

xpos = xpos.flatten()*1./2 #take times 1/2 to get to the center of the bin
ypos = ypos.flatten()*1./2
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = np.zeros_like(xpos)

dx = xedges [1] - xedges [0]
dy = yedges [1] - yedges [0]
dz = jointProb.ravel()

fig, ax = plt.subplots(subplot_kw=dict(projection='3d'), figsize=(8,6))
ax.bar3d(xpos, ypos, zpos, dx, dy, dz, shade=True)

ax.set_xlabel('T1 Intensity')
ax.set_ylabel('T2 Intensity')
ax.set_zlabel('Joint Probability')
plt.show()


### Task 3: Compute the mutual information between the two images
Write a function that takes two image volumes defined on the same image grid as input, and returns the mutual information between the two images:

$$ 
MI = H_{F} + H_{M} - H_{F,M} 
$$
with
$$ 
H_{F,M} = - \sum_{f=1}^{B} \sum_{m=1}^{B} p_{f,m} \log(p_{f,m}),
$$
$$ 
H_{F} = - \sum_{f=1}^{B} p_{f} \log(p_{f}), 
$$
and
$$
H_{M} = - \sum_{m=1}^{B} p_{m} \log(p_{m})
.
$$

Your function should make use of the joint histogram computed as in the previous task (i.e., using $B=32$ bins and ignoring all voxels with intensity lower than 10).

Use your new function to compute the mutual information between the T1-weighted image and the resampled T2-weighted image.

> ***Hints:***
>
> - To avoid numerical errors when one of the bins is empty and 0*log(0)=0 is therefore computed numerically, you can add a tiny value (e.g., 1e-12) to each histogram bin

In [42]:
def mutual_information(B, T1_data, T2_data_resampled, threshold):
    #Finding the joint histogram
    H, x, y = np.histogram2d( T1_data.flatten(), T2_data_resampled.flatten(), bins=B, range=[[threshold, np.percentile(T1_data, 98)], [threshold, np.percentile(T2_data_resampled, 98)]] )
    jointProb = H / np.sum(H)
    H_FM = -np.sum( jointProb * np.log( jointProb + 1e-12 ) ) #adding a small number to avoid log(0)

    #finding the histogram for T1
    H_T1, x = np.histogram( T1_data.flatten(), bins=B, range=(threshold, np.max(T1_data)) )
    p_F = H_T1 / np.sum(H_T1)
    H_F = -np.sum( p_F * np.log( p_F + 1e-12 ) ) #adding a small number to avoid log(0)

    #finding the histogram for T2
    H_T2, x = np.histogram( T2_data_resampled.flatten(), bins=B, range=(threshold, np.max(T2_data_resampled)) )
    p_M = H_T2 / np.sum(H_T2)
    H_M = -np.sum( p_M * np.log( p_M + 1e-12 ) ) #adding a small number to avoid log(0)

    #Calculating the mutual information
    MI= H_F + H_M - H_FM
    return MI

In [43]:
MI= mutual_information(bins, T1_data, T2_re, lb)
print(MI)


-1.6657827518940032


### Task 4: Evaluate the mutual information across a range of rotation angles

Implement a grid search over a range of rotation angles, with the goal of identifying the angle with approximately the highest mutual information (i.e., where the registration is best). Make an educated guess of a suitable range by visually inspecting the images (or by simply trying a few rotation angles and inspecting the result), and then define a list of candidate angles at intervals of e.g., 5° apart:

       a = np.arange(a_{lowest}, a_{highest}, 5)

Loop over all candidate rotation angles, each time (1) creating a corresponding rotation matrix $\mathbf{R}$; (2) resampling the T2-weighted volume accordingly; (3) calculating the mutual information with the function you created above; and (4) storing the mutual information value.

> ***Hint:***
> 
> - A 3D rotation matrix can be parameterized as follows:
> 
> $$\mathbf{R} = \mathbf{R}_{z} \mathbf{R}_{y} \mathbf{R_{x}},$$ 
> where
>
> $$
\mathbf{R}_z = 
\begin{pmatrix} 
 cos( \theta_z ) & -sin( \theta_z ) & 0 & \\
 sin( \theta_z ) & cos( \theta_z ) & 0 & \\
 0 & 0 & 1 & \\
\end{pmatrix}
$$
> implements a rotation around the z-axis in world coordinates;
> $$
\mathbf{R}_y = 
\begin{pmatrix} 
 cos( \theta_y ) & -sin( \theta_y ) & 0 & \\
 0 & 1 & 0 & \\
 sin( \theta_y ) & cos( \theta_y ) & 0 & \\
\end{pmatrix}
$$
> is a rotation around the y-axis;
and
> $$
\mathbf{R}_x = 
\begin{pmatrix} 
 1 & 0 & 0 & \\
 cos( \theta_x ) & -sin( \theta_x ) & 0 & \\
 sin( \theta_x ) & cos( \theta_x ) & 0 & \\
\end{pmatrix}
$$
> rotates around the x-axis.
>
>
> - If you didn't figure out in Task 1 which axis the T2-weighted volume was rotated around, you can try rotating around each axis (one at a time) to see the effect of each transformation. Once you've determined the correct rotation axis, the rotation angles around the other axes should be clamped to zero.


In [44]:
def rotation_matrix(a, i):
    R_z = np.array( [ [ np.cos(np.deg2rad(a[i])), -np.sin(np.deg2rad(a[i])), 0 ],
                    [ np.sin(np.deg2rad(a[i])),  np.cos(np.deg2rad(a[i])), 0 ],
                    [ 0, 0, 1 ] ] )

    R_y = np.array( [ [ np.cos(np.deg2rad(a[i])),0,  -np.sin(np.deg2rad(a[i])),  ],
                    [ 0,  1, 0 ],
                    [ np.sin(np.deg2rad(a[i])),0,  np.cos(np.deg2rad(a[i])) ] ] )
    R_x = np.array( [ [ 1, 0, 0 ],
                    [ 0, np.cos(np.deg2rad(a[i])), -np.sin(np.deg2rad(a[i])) ],
                    [ 0, np.sin(np.deg2rad(a[i])),  np.cos(np.deg2rad(a[i])) ] ] )  
    return R_x, R_y, R_z

In [45]:
def mesh(T1_data, T2_data, M, M_1, M_2):
    V1, V2, V3 = np.meshgrid(np.arange(T1_data.shape[0]),
                         np.arange(T1_data.shape[1]),
                         np.arange(T1_data.shape[2]),
                         indexing= 'ij')
    V1, V2, V3 = V1.flatten(), V2.flatten(), V3.flatten()
    V_T1 = np.column_stack([V1, V2, V3, np.ones(len(V3))])
    V_T2 = np.linalg.solve(M_2, M @ M_1 @ V_T1.T)

    T2_int = map_coordinates(T2_data, V_T2[:3], order = 3)
    T2_re  = T2_int.reshape(T1_data.shape)
    return T2_re

In [46]:
a = np.arange(0, 90, 5)
mi_values = []
image_resampled = []

for i in range(len(a)):
    R_x, R_y, R_z = rotation_matrix(a, i)

    R = R_z

    M = np.eye(4)
    M[:3, :3] = R
    T2_re = mesh(T1_data, T2_data, M, M_1, M_2)
    image_resampled.append(T2_re)

    mi = mutual_information(bins, T1_data, T2_re, lb)
    mi_values.append(mi)

best_fit_idx = np.argmax(mi_values)
best_angle = a[best_fit_idx]
print(f"\nBest rotation angle: {best_angle}° with MI = {mi_values[best_fit_idx]:.4f}")

# --- Plot MI vs angles ---
plt.figure()
plt.plot(a, mi_values, '-o')
plt.xlabel("Rotation angle (°)")
plt.ylabel("Mutual Information")
plt.title("MI vs Rotation Angle")
plt.grid(True)
plt.show()


best_resampled = image_resampled[best_fit_idx]
z = T1_data.shape[2] // 2  # Middle slice
overlay_best = (best_resampled[:, :, z] / best_resampled.max()) + (T1_data[:, :, z] / T1_data.max())

plt.figure(figsize=(8, 8))
plt.imshow(overlay_best.T, cmap='gray', origin='lower')
plt.title(f'Overlay of T1 and Resampled T2 (middle slice) — Best angle: {best_angle}°')
plt.axis('off')
plt.show()

Viewer( best_resampled / best_resampled.max() + T1_data / T1_data.max() )    


Best rotation angle: 45° with MI = -1.4749


### Task 5: Perform automatic registration

Plot the negative mutual information, i.e. the energy function $E(\mathbf{w})= H_{F,M}-H_{F}-H_{M}$ for every angle in your grid search space. Select the one with the lowest energy (i.e., the best angle for registration) and transform the T2-weighted volume according to this angle.

Use the Viewer() function to visualize your registration result.

In [47]:
resampled = []
energi  = []

for i in range(len(a)):
    R_z, _, _ = rotation_matrix(a, i)

    M = np.eye(4)
    M[:3][:3] = R_z

    T2_re = mesh(T1_data, T2_data)
    resampled.append(T2_re)
    mi = mutual_information(bins, T1_data, T2_re, lb)

    energi.append(-mi)
    

ValueError: could not broadcast input array from shape (3,3) into shape (3,4)

### Task 6: Compare the joint histogram before and after registration

Visualize the joint histogram after registration. Compare the result to your initial joint histogram (i.e., before registration) and discuss your findings in your report.

### Additional task for the enthusiastic student: Compare the MI obtained with landmark-based and intensity-based registration

Perform landmark-based registration between the T1-weighted and T2-weighted volumes instead, just like you did in the last exercise. For the resulting registration, view your results with the Viewer() class; visualize the joint histogram, and calculate the mutual information. 

Compare the results with those obtained using mutual information, and discuss.

> ***Hint:***
> - You can reuse the landmarks that you collected in the landmark-based registration exercise, as these were defined in voxel space. 